In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.metrics import roc_curve, auc, precision_recall_curve, average_precision_score
from sklearn.preprocessing import label_binarize
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split
from dataclasses import dataclass, field
from typing import List

## Input staging function Declarations

### Tripeptide Column Generation

In [ ]:
def getTripeptideColumn(df, ordered_list, disordered_list):
    '''
    Input: Dataframe containing the Columns: ID with PDB IDs, Residue Number containing position in the peptide,
           amino_acid containing the AA at that position and Disordered containing a binary value 

           List of Ordered statistically significant tripeptide motifs

           List of Disordered statistically significant tripeptide motifs

    Output: Input dataframe with a tripeptide significance column appended containing an array with binary values
            indicating whether all possible tripeptides containing that positions amino acid statistical significance

    Function: This function runs along every amino acid in a peptide and checks all tripeptides that contain it for 
              statistical significance against the lists provided 
    '''
    Significance_df = pd.DataFrame()
    # Iterate over rows in the data frame
    uniprot_ids = sorted(set(df.loc[:,'ID']))
    for uniprot_id in uniprot_ids:
        uniprot_df = df[df['ID'] == uniprot_id]
        residue_numbers = uniprot_df['residue_number'].values
        for i in uniprot_df.index:
            tripeptides = list()
            if uniprot_df.loc[i, 'Disordered'] == 1:
                tripeptide_list = disordered_list

            elif uniprot_df.loc[i, 'Disordered'] == 0:
                tripeptide_list = ordered_list

            else:
                tripeptide_list = ordered_list
    # First Amino Acid
            if i == uniprot_df.index[0]:
                    """
                    [A]-T-C-Y-K-P-Y-L
                         0  1 2 3 4 5 6 7
                    
                    Should only return ATC
                    
                    """
                    tripeptide = uniprot_df.loc[i:i+2,'amino_acid'].values
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.at[i, 'tripeptide_significance'] = [0,0,1]

                        else:
                            uniprot_df.at[i, 'tripeptide_significance'] = [0,0,0]

    # Second Amino Acid
            elif i == uniprot_df.index[1]:
                    """ 
                    A-[T]-C-Y-K-P-Y-L
                         0  1  2 3 4 5 6 7
                    
                    Should only return ATC and TCY
                    
                    """
                    
                    tripeptide = uniprot_df.loc[i-1:i+1,'amino_acid'].values
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.at[i, 'tripeptide_significance'] = [0,1]

                        else:
                            uniprot_df.at[i, 'tripeptide_significance'] = [0,0]
                    
                    tripeptide = uniprot_df.loc[i:i+2,'amino_acid'].values
                    
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(1)

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)

    # Last Amino Acid
            elif i == uniprot_df.index[-1]:
                    
                    """  A-T-C-Y-K-P-Y-[L]
                         0 1 2 3 4-3-2 -1
                         
                    Should only return PYL
                    
                    """
                    tripeptide = uniprot_df.loc[i-2:i,'amino_acid'].values
                    
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.at[i, 'tripeptide_significance'] = [1,0,0]

                        else:
                            uniprot_df.at[i, 'tripeptide_significance'] = [0,0,0]

    # Second Last Amino Acid
            elif i == uniprot_df.index[-2]:
                    
                    """  A-T-C-Y-K-P-[Y]-L
                         0 1 2 3 4-3 -2 -1
                         
                    Should only return PYL and KPY
                    
                    """
                    tripeptide = uniprot_df.loc[i-2:i,'amino_acid'].values
                    
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'] = [1]

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'] = [0]
                    
                    tripeptide = uniprot_df.loc[i-1:i+1,'amino_acid'].values
                    
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(1)
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)
                            

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)
    # All Other AAs
            else: 
                    
                    tripeptide = uniprot_df.loc[i-2:i,'amino_acid'].values
                    
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'] = [1]

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'] = [0]
                        
                    tripeptide = uniprot_df.loc[i-1:i+1,'amino_acid'].values
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(1)

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)
                        
                    tripeptide = uniprot_df.loc[i:i+2,'amino_acid'].values
                    if len(tripeptide) != 3: 
                        sys.stderr.write(f"{tripeptide}\tERROR!!!! at position {i} in {uniprot_df.loc[i, 'ID']}\n")
                    else:
                        tripeptide = [''.join(tripeptide)]
                        if tripeptide in tripeptide_list:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(1)

                        else:
                            uniprot_df.loc[i, 'tripeptide_significance'].append(0)
        
        Significance_df = pd.concat([Significance_df ,uniprot_df], ignore_index=True)

    return(Significance_df)
                        

### Amino Acid Vector Generation

In [ ]:
def getAA_column(df):
    '''
    Input: Dataframe containing the Columns: ID with PDB IDs, Residue Number containing position in the peptide,
           amino_acid containing the AA at that position 

    Output: Input Dataframe with column containing a vector representing that rows amino acid in one hot encoding

    Function: Take all rows amino acid a generate a vector representing that amino acid in one hot encoding 
    '''

    vectorised_df = pd.DataFrame()
    # Iterate over rows in the data frame
    uniprot_ids = sorted(set(df.loc[:,'ID']))
    for uniprot_id in uniprot_ids:
        uniprot_df = df[df['ID'] == uniprot_id]
        residue_numbers = uniprot_df['residue_number'].values
        for i in uniprot_df.index:
    # Positively Charged AAs
    # Arginine - Pos 1
            if uniprot_df.loc[i, 'amino_acid'] == 'R':
                i_list = [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Histidine - Pos 2
            elif uniprot_df.loc[i, 'amino_acid'] == 'H':
                i_list = [0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector
                
    # Lysine - Pos 3
            elif uniprot_df.loc[i, 'amino_acid'] == 'K':
                i_list = [0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Negatively Charged AAs
    # Aspartic Acid - Pos 4
            elif uniprot_df.loc[i, 'amino_acid'] == 'D':
                i_list = [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Glutamic Acid - Pos 5
            elif uniprot_df.loc[i, 'amino_acid'] == 'E':
                i_list = [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Polar Uncharged AAs
    # Serine - Pos 6
            elif uniprot_df.loc[i, 'amino_acid'] == 'S':
                i_list = [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Threonine - Pos 7 
            elif uniprot_df.loc[i, 'amino_acid'] == 'T':
                i_list = [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Asparagine  - Pos 8 
            elif uniprot_df.loc[i, 'amino_acid'] == 'N':
                i_list = [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Glutamine - Pos 9 
            elif uniprot_df.loc[i, 'amino_acid'] == 'Q':
                i_list = [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Special cases - come back to this
    # Glycine - Pos 10
            elif uniprot_df.loc[i, 'amino_acid'] == 'G':
                i_list = [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Proline - Pos 11
            elif uniprot_df.loc[i, 'amino_acid'] == 'P':
                i_list = [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Cysteine - Pos 12
            elif uniprot_df.loc[i, 'amino_acid'] == 'C':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Hydrophobic AAs
    # Alanine - Pos 13
            elif uniprot_df.loc[i, 'amino_acid'] == 'A':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Valine - Pos 14
            elif uniprot_df.loc[i, 'amino_acid'] == 'V':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Isoleucine - Pos 15
            elif uniprot_df.loc[i, 'amino_acid'] == 'I':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Leucine - Pos 16
            elif uniprot_df.loc[i, 'amino_acid'] == 'L':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Methionine - Pos 17
            elif uniprot_df.loc[i, 'amino_acid'] == 'M':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Phenylalanine - Pos 18
            elif uniprot_df.loc[i, 'amino_acid'] == 'F':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector
        
    # Tyrosine - Pos 19
            elif uniprot_df.loc[i, 'amino_acid'] == 'Y':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Tryptophan - Pos 20
            elif uniprot_df.loc[i, 'amino_acid'] == 'W':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector

    # Empty Position - Pos Null
            elif uniprot_df.loc[i, 'amino_acid'] == '-':
                i_list = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
                i_vector = np.array(i_list)
                uniprot_df.at[i, 'amino_acid_vector'] = i_vector


            else:
                print("Mahoraga HELP ME")

        vectorised_df = pd.concat([vectorised_df ,uniprot_df], ignore_index=True)
    return vectorised_df

### Generate Ground truth binding binary value

In [ ]:
def getGT_column(df):
    '''
    Input: Dataframe containing the Columns: ID with PDB IDs, Residue Number containing position in the peptide
           and a column containing a givens rows amino acids distance to RNA

    Output: Input dataframe with column appended containing binary binding value

    Function: Take the distance to RNA column and generate a binding binary vector based on whether its below 
              the threshold of 4.2
    '''
    GT_df = pd.DataFrame()
    # Iterate over rows in the data frame
    uniprot_ids = sorted(set(df.loc[:,'ID']))
    for uniprot_id in uniprot_ids:
        uniprot_df = df[df['ID'] == uniprot_id]
        residue_numbers = uniprot_df['residue_number'].values
        for i in uniprot_df.index:
            if uniprot_df.loc[i, 'Distance_to_RNA'] > 4.2:
                uniprot_df.at[i, 'binding_ground_truth'] = [0]

            elif uniprot_df.loc[i, 'Distance_to_RNA'] <= 4.2:
                uniprot_df.at[i, 'binding_ground_truth'] = [1]

            else:
                sys.stderr.write(f"Non-Integer Input within Distance_to_RNA column")

        GT_df = pd.concat([GT_df, uniprot_df], ignore_index=True)

    return GT_df
            

### Disorder Prediction using Albatross and Sparrow Package

### Generate Dataframe for Model training

In [ ]:
def training_data_preprocess(df, ordered_list, disordered_list, do_GT):
    
    if do_GT == True:
        req_columns = ["No", "ID", 'pdb_id', 'residue_number', 'amino_acid', 'Distance_to_RNA']

    elif do_GT == False:
        req_columns = ["No", "ID", 'residue_number', 'amino_acid', 'Disordered', 'RNA_binding']

    tripeptide_ready_data = df[req_columns]
    tripeptide_ready_data['tripeptide_significance'] = pd.NA
    tripeptide_ready_data['amino_acid_vector'] = pd.NA
    tripeptide_ready_data['binding_ground_truth'] = pd.NA

    post_tripeptide_df = getTripeptideColumn(tripeptide_ready_data, ordered_list, disordered_list)

    aa_vectorised_df = getAA_column(post_tripeptide_df)

    if do_GT == True:
        finalised_gt_df = getGT_column(aa_vectorised_df)

    if do_GT == False:
        finalised_gt_df = aa_vectorised_df

    resorted_finalsed_df = finalised_gt_df.sort_values(by='No', ascending = True)

    if do_GT == True:
        final_columns = ['ID','amino_acid_vector', 'tripeptide_significance', 'binding_ground_truth']

    if do_GT == False:
        final_columns = ['ID', 'amino_acid_vector', 'tripeptide_significance', 'Disordered', 'RNA_binding']
    
    model_input_final = resorted_finalsed_df[final_columns]

    return model_input_final

### Input Vector Staging

In [ ]:
training_data = pd.read_csv('All_combined_results.txt', sep='\t')

In [ ]:
top20_tripep_ordered = pd.read_csv('Top20_tripeptides_ordered.csv')
top20_tripep_disordered = pd.read_csv('Top20_tripeptides_disordered.csv')

top20_tripep_ordered.rename(columns={'0': 'tripeptides'}, inplace= True)
top20_tripep_disordered.rename(columns={'0': 'tripeptides'}, inplace= True)

diso_significant_tripeptides = top20_tripep_disordered['tripeptides'].values
diso_significant_tripeptides = diso_significant_tripeptides.tolist()

ordered_significant_tripeptides = top20_tripep_ordered['tripeptides'].values
ordered_significant_tripeptides = ordered_significant_tripeptides.tolist()


In [ ]:
completed_input_df = training_data_preprocess(training_data, significant_tripeptides, True)
completed_input_df['binding_ground_truth'] = completed_input_df['binding_ground_truth'].apply(lambda x: x[0])

In [ ]:
# Assuming 'amino_acid_vector' needs to be expanded
amino_acid_df = pd.DataFrame(completed_input_df['amino_acid_vector'].tolist(), index=completed_input_df.index)
amino_acid_df.columns = [f'amino_acid_{i}' for i in range(1,len(amino_acid_df.columns)+1)]  # Optional: Naming columns

tripeptide_df = pd.DataFrame(completed_input_df['tripeptide_significance'].tolist(), index=completed_input_df.index)
tripeptide_df.columns = [f'tri_peptide_aa_{i}' for i in range(1,len(tripeptide_df.columns)+1)]  # Optional: Naming columns

# Now join this back to the original dataframe if necessary
df_expanded = completed_input_df.join(amino_acid_df)
df_expanded = df_expanded.join(tripeptide_df)
df_expanded.drop(columns=['amino_acid_vector','tripeptide_significance'], axis=1, inplace=True)

In [ ]:
df_expanded

In [ ]:
df_expanded.to_csv('Training_data.txt',sep='\t')

### Import of Disordered Test and Train Data

In [ ]:
def read_in_diso_data(training_data_path):
    rows = []
    with open(training_data_path, 'r') as file:
        data = file.read()
        lines = data.strip().split('\n')
        for i in range(0, len(lines), 6):
            PDB_ID = lines[i].strip('>')
            Protein_Seq = lines[i+1]
            Disordered = lines[i+2]
            RNA_binding = lines[i+5]

            for i in range(len(Protein_Seq)):
                rows.append({'ID': PDB_ID, 'residue_number': i+1, 'amino_acid': Protein_Seq[i], 'Disordered': int(Disordered[i]), 'RNA_binding': int(RNA_binding[i])})

    staged_data_df = pd.DataFrame(rows)

    staged_data_df = staged_data_df[staged_data_df['Disordered'] != 0]

    staged_data_df['No'] = range(len(staged_data_df))

    staged_data_df = staged_data_df.reset_index()

    return staged_data_df

In [ ]:
diso_training_data_df = read_in_diso_data('Diso_TrainingDataset.txt')

In [ ]:
diso_test_data_df = read_in_diso_data('TestDataset.txt')